**Smoke-Fire-Detection-YoLo11-Model**

**Step 01 # Install the Ultralytics Package**

In [ ]:
!nvidia-smi
!pip install ultralytics
!pip install roboflow

**Step 02 # Import All the Requried Libraries**

In [ ]:
import ultralytics
from ultralytics import YOLO
from IPython.display import Image
from roboflow import Roboflow
from kaggle_secrets import UserSecretsClient
import os

ultralytics.checks()

**Step # 03 Download Dataset from Roboflow**

In [ ]:
# import shutil

# shutil.rmtree('/kaggle/working/Fire-and-Smoke-Dataset-2')

In [ ]:
user_secrets = UserSecretsClient()
robo_key = user_secrets.get_secret("ROBOFLOW_KEY")


rf = Roboflow(api_key=robo_key)
project = rf.workspace("sayed-gamall").project("fire-smoke-detection-yolov11")
version = project.version(2)
dataset = version.download("yolov11")

In [ ]:
dataset.location

#### Path correction

In [ ]:
import yaml

# Path to the YAML file
file_path = f'{dataset.location}/data.yaml'

# Step 1: Read the YAML file
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

# Step 2: Update paths for train, val, and test
base_path = dataset.location
data['train'] = f"{base_path}/train/images"
data['val'] = f"{base_path}/valid/images"
data['test'] = f"{base_path}/test/images"

# Step 3: Save the updated YAML data back to the file
with open(file_path, 'w') as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("Paths updated successfully!")

#### Verification

In [ ]:
from pprint import pprint
with open(file_path, 'r') as file:
    updated_data = yaml.safe_load(file)
    pprint(updated_data)

### Labels Clearning

In [ ]:
import os
from collections import defaultdict

def read_yolo_annotations_with_metadata(dataset_metadata):
    # Extract paths and class names
    train_path = dataset_metadata['train'].replace('images', 'labels')
    val_path = dataset_metadata['val'].replace('images', 'labels')
    test_path = dataset_metadata['test'].replace('images', 'labels')
    class_names = dataset_metadata['names']
    
    # Storage variables
    split_stats = {
        'train': defaultdict(int),
        'val': defaultdict(int),
        'test': defaultdict(int)
    }

    def process_labels(label_dir, split_name):
        for label_file in os.listdir(label_dir):
            if not label_file.endswith(".txt"):
                continue

            label_path = os.path.join(label_dir, label_file)
            
            # Read annotations
            with open(label_path, 'r') as f:
                lines = [line.strip() for line in f.readlines() if line.strip()]
            
            # Find present classes
            present_classes = set()
            for line in lines:
                class_id = int(line.strip().split()[0])
                present_classes.add(class_id)
            
            # Update counts
            if 0 in present_classes and 1 in present_classes:
                split_stats[split_name]['both'] += 1
            elif 0 in present_classes:
                split_stats[split_name]['fire_only'] += 1
            elif 1 in present_classes:
                split_stats[split_name]['smoke_only'] += 1
            else:
                split_stats[split_name]['background'] += 1

    # Process all datasets
    process_labels(train_path, 'train')
    process_labels(val_path, 'val')
    process_labels(test_path, 'test')

    return {
        'class_stats': split_stats,
        'class_names': class_names
    }

# Example usage
dataset_metadata = {
    'names': ['Fire', 'Smoke'],
    'nc': 2,
    'test': '/kaggle/working/Fire-Smoke-Detection-Yolov11-2/test/images',
    'train': '/kaggle/working/Fire-Smoke-Detection-Yolov11-2/train/images',
    'val': '/kaggle/working/Fire-Smoke-Detection-Yolov11-2/valid/images'
}

results = read_yolo_annotations_with_metadata(dataset_metadata)

# Print the report
print("Fire/Smoke Distribution Across Splits:\n")
for split in ['train', 'val', 'test']:
    stats = results['class_stats'][split]
    total = sum(stats.values())
    print(f"{split.upper()} SET ({total} images)")
    print(f"- Fire only: {stats['fire_only']} ({stats['fire_only']/total:.1%})")
    print(f"- Smoke only: {stats['smoke_only']} ({stats['smoke_only']/total:.1%})")
    print(f"- Both present: {stats['both']} ({stats['both']/total:.1%})")
    print(f"- Background (neither): {stats['background']} ({stats['background']/total:.1%})")
    print()

**Step # 04 Train YOLO11 Model on a Custom Dataset**

In [ ]:
!yolo task=detect mode=train data={dataset.location}/data.yaml model="yolo11n.pt" epochs=250 imgsz=640 batch=32 patience=20

**Step # 05 Examine Training Results**

In [ ]:
Image("/kaggle/working/runs/detect/train/confusion_matrix.png", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/labels.jpg", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/results.png", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/train_batch0.jpg", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/val_batch0_pred.jpg", width=600)

In [ ]:
Image("/kaggle/working/runs/detect/train/val_batch1_pred.jpg", width=600)

**Step # 07  Validate Fine-Tuned Model**

In [ ]:
!yolo task=detect mode=val model="/kaggle/working/runs/detect/train/weights/best.pt" data={dataset.location}/data.yaml

**Step # 08 Inference with Custom Model on Images**

In [ ]:
!yolo task=detect mode=predict model="/kaggle/working/runs/detect/train/weights/best.pt" conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob('/kaggle/working/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[1:4]:
    display(IPyImage(filename=img, width=600))
    print("\n")